In [1]:
from __future__ import division
from aqchem.chemistry import Solute
from aqchem.equilibria import Equilibrium, EqSystem, charge_balance, atom_balance, REqSystem
import periodictable
import numpy as np

In [2]:
substances = Hp, OHm, NH4p, NH3, H2O = [
    Solute(n, formula=periodictable.formula(n)) for n in [
        'H{+}', 'HO{-}', 'NH3 + H{+}', 'NH3', 'H2O']]
#substances = substances[::-1]
assert (Hp.charge, OHm.charge, NH4p.charge, NH3.charge, H2O.charge) == (1, -1, 1, 0, 0)
init_conc = {Hp: 1e-7, OHm: 1e-7, NH4p: 1e-7, NH3: 1.0, H2O: 55.5}
x0 = [init_conc[k] for k in substances]
H2O_c = init_conc[H2O]
w_autop = Equilibrium({H2O: 1}, {Hp: 1, OHm: 1}, 10**-14/H2O_c)
NH4p_pr = Equilibrium({NH4p: 1}, {Hp: 1, NH3: 1}, 10**-9.26)
equilibria = w_autop, NH4p_pr
init_conc

AttributeError: 'Solute' object has no attribute 'anme'

In [ ]:
reqsys = REqSystem(equilibria, substances)
reqsys.root(init_conc, scaling=1.0)

In [ ]:
reaction_system = EqSystem(equilibria, substances)

In [ ]:
import sympy as sp
sp.init_printing()
ny = len(substances)
y = sp.symarray('y', ny)
i = sp.symarray('i', ny)
Kw, Ka, sigma = sp.symbols('K_w K_a sigma')
w_autop.params = Kw
NH4p_pr.params = Ka
rs = EqSystem(equilibria, substances)
#f, elim, red_cbs = rs.f(y, i, scaling=sigma, pres1st=True, presw=1000, norm=True, const_indices=(4,))
ss = sp.symarray('s', ny)
ms = sp.symarray('m', ny)
f, elim, red_cbs = rs.f([_*sigma for _ in y], i, scaling=sigma, norm=True, pres_norm=True)
#tanh_subs = [(yi, m + s*sp.tanh((yi - m)/s)) for
#             yi, m, s in zip(y, ms, ss)]
#f = [_.subs(tanh_subs) for _ in f]
f

In [ ]:
j = sp.Matrix(1, len(f), lambda _, q: f[q]).jacobian(y[:-1])
init_conc_j = {Hp: 1e-10, OHm: 1e-7, NH4p: 1e-7, NH3: 1.0, H2O: 55.5}
jarr = np.array(j.subs(zip(y,x0)).subs({sigma: 1e0, Kw: 1e-14, Ka: 10**-9.26}).subs(
            zip(i, reaction_system.as_per_substance_array(init_conc_j))))
jarr = np.asarray(jarr, dtype=np.float64)
np.log10(np.linalg.cond(jarr))

In [ ]:
j

In [ ]:
f, elim, red_cbs = rs.f(y, i, norm=True, reduced=True)
f, elim

In [ ]:
[s.name for s in rs.substances]

In [ ]:
rs.charge_balance_vector(), rs.atom_balance_vectors()

In [ ]:
rs.rref()

In [ ]:
np.set_printoptions(4, linewidth=120)
scaling = 1e8
for rxn in rs.rxns:
    rxn.params = rxn.params.subs({Kw: 1e-14, Ka: 10**-9.26})

In [ ]:
x, res = rs.root(x0, scaling=1, logC=False, square=False)
res.success

In [ ]:
x, res = rs.root({Hp: 1e-11, OHm: 1e-3, NH4p: 1e-3, NH3: 1.0, H2O: 55.5}, scaling=1, logC=False, square=False)
res.success

In [ ]:
x, res = rs.root({Hp: 1.7e-11, OHm: 3e-2, NH4p: 3e-2, NH3: 0.97, H2O: 55.5}, scaling=1e8, logC=False, square=False)
res.success

In [ ]:
x, res = rs.root({Hp: 1.7e-11, OHm: 3e-2, NH4p: 3e-2, NH3: 0.97, H2O: 55.5}, scaling=1e16, logC=True, square=False)
x

In [ ]:
rs.root(x0, scaling=1e8, logC=False, square=False)

In [ ]:
rs.root(x0, scaling=1, logC=False, square=False)

In [ ]:
rs.root(x0, scaling=1.0, logC=False, reduced=True)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
ny = len(rs.substances)
nc = 10
x = np.empty((nc, ny))
Hparr = np.logspace(-4, 0, nc)
success = []
for idx in range(nc):
    x00 = x0[:]
    x00[0] = Hparr[idx]
    resx, res = rs.root(x00, scaling=1e12)
    success.append(res.success)
    x[idx, :] = resx
for idx_s in range(ny):
    plt.loglog(Hparr, x[:, idx_s], label=rs.substances[idx_s].name)
plt.legend(loc='best')
for i, s in enumerate(success):
    if s is False:
        plt.axvline(Hparr[i], c='k', ls='--')

In [ ]:
init_conc

In [ ]:
nc=30
fig = plt.figure(figsize=(16, 6))
ax1 = plt.subplot(2, 3, 1, xscale='log', yscale='log')
Cout_1, ic1, success1 = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e14, ax=ax1)
ax2 = plt.subplot(2, 3, 2, xscale='log', yscale='log')
Cout_2, ic2, success2 = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e14, const=(H2O,), ax=ax2)
ax3 = plt.subplot(2, 3, 3, xscale='log', yscale='log')
Cout_3, ic3, success3 = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e14, method='lm', ax=ax3)
ax4 = plt.subplot(2, 3, 4, xscale='log', yscale='log')
Cout_4, ic4, success4 = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e14, reduced=True, ax=ax4)
ax5 = plt.subplot(2, 3, 5, xscale='log', yscale='log')
Cout_5, ic5, success5 = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e14, reduced=True, norm=True, ax=ax5)
all(success1), all(success2), all(success3), all(success4)

In [ ]:
rs.plot_errors(Cout_1, ic1, Hp)

In [ ]:
Cout_14, ic, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e6, pres1st=True)
all(success)

In [ ]:
Cout_14, ic, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e6, pres1st=True, norm=True, method='lm')
all(success)

In [ ]:
Cout_14, ic, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e6, pres1st=True, norm=True, init_iter=15)
all(success)

In [ ]:
nc = 40
Cout_14, ic, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e3,
                                pres1st=True, norm=True, init_iter=10, presw=10000)#, carry=True)
all(success)

In [ ]:
Cout, inits, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e6, pres1st=True, presw=1, norm=True, init_iter=15)
all(success)

In [ ]:
import random
subst = tuple(rs.substances)
plt.figure(figsize=(16,6))
for i in range(1, 2*3+1):
    ax = plt.subplot(2, 3, i, xscale='log', yscale='log')
    Cout, ic, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, 30), scaling=1e14, ax=ax)
    plt.title(', '.join([str(s) for s in rs.substances]))
    random.shuffle(rs.substances)
rs.substances = list(subst)
init_conc

In [ ]:
Cout_14, ic, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e14, norm=True)
all(success)

In [ ]:
Cout_logC, inits_out, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e3, logC=True,
                                        presw=10000)
all(success)

In [ ]:
rs.plot_errors(Cout_logC, inits_out, Hp)

In [ ]:
Cout_logC, inits_out, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), logC=True, init_guess=Cout_14, init_iter=0)
all(success)

In [ ]:
Cout_14, ic14, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e14, carry=True)
all(success)

In [ ]:
_1, _2, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e-4, tol=1e-12, init_iter=100)
all(success)

In [ ]:
_1, _2, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e-8)
all(success)

In [ ]:
_1, _2, success = rs.plot(init_conc, Hp, np.logspace(-4, 0, nc), scaling=1e-8, carry=True)
all(success)

In [ ]:
Cout, inits_out, success = rs.plot(Cout_14, Hp, np.logspace(-4, 0, nc), scaling=1, init_iter=0, method='lm')
all(success), Cout.shape

In [ ]:
Cout, inits_out, success = rs.plot(Cout_14, Hp, np.logspace(-4, 0, nc), scaling=1, logC=True, init_iter=0, method='lm')
all(success)

In [ ]:
rs.plot_errors(Cout, inits_out, Hp)

In [ ]:
Cout, inits_out, success = reqsys.plot(init_conc, Hp, np.logspace(-7, 0, 20), scaling=1, carry=True)

In [ ]:
x0

In [ ]:
from aqchem.equilibria import solve_equilibrium
print(x0)
for w in range(1,5):
    for eq in rs.rxns:
        new_x0 = solve_equilibrium(x0, eq.net_stoich(rs.substances), eq.params)
        x0 = (w*x0 + new_x0)/(w+1)
        print(x0)